In [0]:
import mlflow
import mlflow.spark
from mlflow.exceptions import RestException
from pyspark.ml.feature import StringIndexer
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
test = spark.table("MLOps.data.test_table")

model_name = "als_recommendation_model"
experiment_path = "/Workspace/Users/jung@ap-com.co.jp/mlops_demo_model/als_recommendation"
experiment = mlflow.get_experiment_by_name(experiment_path)
runs_df = mlflow.search_runs(
    experiment_ids=[experiment.experiment_id]
)
training_runs = runs_df[runs_df["tags.mlflow.runName"] == "training"]
latest_training_run = training_runs.sort_values(
    by="start_time", ascending=False
).iloc[0]
model_uri = f"runs:/{latest_training_run.run_id}/als_model"

challenger = mlflow.spark.load_model(model_uri)

try:
    champion = mlflow.spark.load_model(f"models:/{model_name}/Production")
    print("Champion model exists in Production")
except Exception as e:
    champion = None
    print("No Champion model found. Challenger will be promoted directly.", e)

evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="interaction_weight",  
    predictionCol="prediction"
)

predictions_challenger = challenger.transform(test)
challenger_rmse = evaluator.evaluate(predictions_challenger)
print("Challenger RMSE:", challenger_rmse)

if champion is not None:
    predictions_champion = champion.transform(test)
    champion_rmse = evaluator.evaluate(predictions_champion)
    print("Champion RMSE:", champion_rmse)
else:
    champion_rmse = None

if champion is None or challenger_rmse < champion_rmse:
    print("Challenger outperforms Champion (or no Champion exists). Promoting...")

    mlflow.register_model(model_uri, model_name)
    client = mlflow.tracking.MlflowClient()

    latest_version = client.get_latest_versions(model_name, stages=["None"])[-1].version

    client.transition_model_version_stage(
        name=model_name,
        version=latest_version,
        stage="Production",
        archive_existing_versions=True
    )
    print("Challenger promoted to Champion (Production).")
else:
    print("Challenger did not outperform Champion. No promotion.")
